In [ ]:
import os
import zipfile
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.metrics import classification_report, confusion_matrix
from google.colab import files
import matplotlib.pyplot as plt

uploaded = files.upload()
zip_file_path = list(uploaded.keys())[0]

dataset_path = "./dataset"
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(dataset_path)

train_path = os.path.join(dataset_path, 'train')
val_path = os.path.join(dataset_path, 'val')
test_path = os.path.join(dataset_path, 'test')

img_size = (150, 150)
batch_size = 32

data_gen_args = {
    "rescale": 1.0 / 255.0,
    "rotation_range": 20,
    "width_shift_range": 0.2,
    "height_shift_range": 0.2,
    "shear_range": 0.2,
    "zoom_range": 0.2,
    "horizontal_flip": True
}
train_gen = ImageDataGenerator(**data_gen_args).flow_from_directory(
    train_path, target_size=img_size, batch_size=batch_size, class_mode='categorical'
)
val_gen = ImageDataGenerator(rescale=1.0 / 255.0).flow_from_directory(
    val_path, target_size=img_size, batch_size=batch_size, class_mode='categorical'
)
test_gen = ImageDataGenerator(rescale=1.0 / 255.0).flow_from_directory(
    test_path, target_size=img_size, batch_size=batch_size, class_mode='categorical', shuffle=False
)

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_size[0], img_size[1], 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(train_gen.num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

model.fit(train_gen, validation_data=val_gen, epochs=20)

eval_loss, eval_acc = model.evaluate(test_gen)
print(f"\nTest Loss: {eval_loss}\nTest Accuracy: {eval_acc}")

preds = np.argmax(model.predict(test_gen), axis=1)
true_labels = test_gen.classes
print("\nClassification Report:")
print(classification_report(true_labels, preds, target_names=list(test_gen.class_indices.keys())))
print("\nConfusion Matrix:")
print(confusion_matrix(true_labels, preds))

model.save("image_classification_model.h5")

def predict_single_image(model, img_path, class_indices, img_size=(150, 150)):
    img = image.load_img(img_path, target_size=img_size)
    img_array = image.img_to_array(img)
    img_array = img_array / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions, axis=1)[0]

    class_labels = {v: k for k, v in class_indices.items()}
    predicted_label = class_labels[predicted_class]

    plt.imshow(img)
    plt.axis('off')
    plt.title(f"Prediction: {predicted_label}")
    plt.show()

    return predicted_label

test_image_path = input("Enter the path of an image to test: ")
predicted_label = predict_single_image(model, test_image_path, test_gen.class_indices)
print(f"Predicted Label: {predicted_label}")
